This worked in web chatgpt:

You are given a json of format {id: ["issue1", "issue2", ....]}

I want you to cluster those issues into overarching critique points. Respond with:

[{"name": <name of cluster>, "description": "<brief description>", "ids": [{id: <critique id>, issues: [<list of issues from that critique that were associated with this cluster. Not all issues mentioned might belong to the same cluster>]}]}]